# Packages

In [ ]:
using LinearAlgebra
using DifferentialEquations, COVID19SchoolReopening
using PyPlot, LaTeXStrings
using ProgressMeter

const CSR = COVID19SchoolReopening

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["axes.grid"] = true
rcParams["font.size"] = 12

# Model Parameters

In [ ]:
# length of simulation, translated from months to days (model time scale)
months = 6
days = months * CSR.DAYS_PER_MONTH
weeks = Int(round(Int, days / 7))
tspan = (0, days*1.0);

In [ ]:
# assuming model time in days

# cohort interaction, between 0 and 1
α = 0.05 * ones(3, 3)
α[1,1] = 1      # cohort 1
α[2,2] = 1      # cohort 2
α[3,3] = 1      # cohort 3

# latent period, time between exposure time and beginning of infectious period
σ = zeros(2)
σ[1] = 1/(3)     # children, 3 days
σ[2] = 1/(3)     # adults, 3 days

# removal rate, 'recovery' from infectious state to non-infectious
γ = zeros(2)
γ[1] = 1/(4)    # child recovery rate, 4 days
γ[2] = 1/(4)    # adult recovery rate, 4 days

# baseline transmission rates
β0 = 1.2
F = [
    0.1     0.25    # child to j
    0.15    0.5     # adult to j
]
β = β0 * F

# population: [children, adults]
demographics = [22., 78.]
total_infected = 2.0
infected = [10., 90.]

params = (α, σ, γ);

In [ ]:
function get_compartment(u, i, j)
    c = zero(eltype(u))

    # iterate over cohorts
    for k in 1:size(u, 3)
        c += max(0, u[i,j,k])
    end
    return c
end

susceptible_child_fraction(u) = get_compartment(u, 1, 1)
infected_child_fraction(u) = get_compartment(u, 3, 1)
removed_child_fraction(u) = get_compartment(u, 4, 1)
cumulative_child_fraction(u) = infected_child_fraction(u) + removed_child_fraction(u)

susceptible_adult_fraction(u) = get_compartment(u, 1, 2)
infected_adult_fraction(u) = get_compartment(u, 3, 2)
removed_adult_fraction(u) = get_compartment(u, 4, 2)
cumulative_adult_fraction(u) = infected_adult_fraction(u) + removed_adult_fraction(u)

# Fig 3: Interaction between multiplier, initial infected, and cohort strategy

Multiple cohorts, increased contact rates on school days (see `TransmissionRate`).

### Testing policy

- Test every day, at the beginning of the day.
- Testing applies only to children attending school that day.
- Identified cases become removed as determined by test sensitivity.

In [ ]:
# testing sensitivity, policy threshold, and window size
sensitivity = 1.0
threshold = 0.05
window = 14

# assume order of magnitude increase in child-child contact
multiplier = 10.0

# testing times - school days only; applies to active cohort
school_days = 1:1.0:5
test_times = [(day-1) + 7*(week-1) + 1 for week in 1:weeks for day in school_days]

**Note**: This assumes a population mix of 22% school-age children.

In [ ]:
infected_range = range(0.0, 10.0, length=201)
multiplier_range = [1, 2, 10]

xrange = multiplier_range
yrange = infected_range

m = length(yrange)
n = length(xrange)

nclasses = 2
ncohorts = 3

t_thresh = [zeros(m, n) for _ in 1:ncohorts+1]

In [ ]:
# scenario parameters
isrotating = true

for k in 1:ncohorts
    # generate problems with different parameter values
    prob_matrix = Array{Any}(undef, m, n)

    for (j, mult) in enumerate(xrange), (i, total_infected) in enumerate(yrange)
        beta = β0 * F
        prob_matrix[i,j] = make_model_instance(nclasses, k, tspan, beta, params,
            mult=mult,
            demographics=demographics,
            total_infected=total_infected,
            infected=infected,
        )
    end

    #
    # solve the ODE for different parameter values
    # save data for each day
    #
    @showprogress "Simulation for $(k) cohort(s)... " for (ix, prob) in enumerate(prob_matrix)
        detected = sizehint!(Float64[], length(test_times))
        cb_test = make_testing_policy_callback(test_times, detected,
            sensitivity=sensitivity,
            isrotating=isrotating,
        )
        sol = solve(prob, callback=cb_test, userdata=Dict(:school_closed=>false))

        # extract 14-day window infection data
        offset = window-1
        x = [ sum(detected[max(1,i-window):i]) for i in eachindex(detected) ]

        # find time to hit threshold
        index = findfirst(≥(demographics[1]/sum(demographics)*threshold), x)
        t_thresh[k][ix] = isnothing(index) ? tspan[end] : test_times[index]
    end
end

In [ ]:
isrotating = false

# generate problems with different parameter values
prob_matrix = Array{Any}(undef, m, n)

for (j, mult) in enumerate(xrange), (i, total_infected) in enumerate(yrange)
    beta = β0 * F
    prob_matrix[i,j] = make_model_instance(nclasses, 2, tspan, beta, params,
        mult=mult,
        demographics=demographics,
        total_infected=total_infected,
        infected=infected,
    )
end

#
# solve the ODE for different parameter values
# save data for each day
#
@showprogress "Simulation for 2 parallel cohorts... " for (ix, prob) in enumerate(prob_matrix)
    # only the first cohort attends school in person
    cohort_β = prob.p[4]

    # cohort 1 --- in-person
    ratef = cohort_β[1][1,1]
    cohort_β[1][1,1] = TransmissionRate(ratef.cmax, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

    # cohort 2 --- remote learning
    ratef = cohort_β[2][1,1]
    cohort_β[2][1,1] = TransmissionRate(ratef.cmin, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

    # run the simulation
    detected = sizehint!(Float64[], length(test_times))
    cb_test = make_testing_policy_callback(test_times, detected,
        sensitivity=sensitivity,
        isrotating=isrotating,
    )
    sol = solve(prob, callback=cb_test, userdata=Dict(:school_closed=>false))

    # extract 14-day window infection data
    offset = window-1
    x = [ sum(detected[max(1,i-window):i]) for i in eachindex(detected) ]

    # find time to hit threshold
    index = findfirst(≥(demographics[1]/sum(demographics)*threshold), x)
    t_thresh[4][ix] = isnothing(index) ? tspan[end] : test_times[index]
end

In [ ]:
# background
rcParams["axes.grid"] = false
rcParams["axes.facecolor"] = "0.90"

fig, axes = subplots(nrows=2, ncols=2, figsize=[8, 8])
linestyle = ["-", "--", ":"]
titlestr = ["1 cohort", "2 cohorts (rotating)", "3 cohorts", "2 cohorts (parallel)"]

reindex = [1,3,2,4]

for (k, ax) in enumerate(axes)
    for j in 1:length(multiplier_range)
        data = @views t_thresh[reindex[k]][:,j] ./ 7
        ix = findfirst(<(26), data)
        ax.plot(infected_range, data, linestyle=linestyle[j], lw=2.5)
        # if !isnothing(ix)
        #     ax.scatter(infected_range[ix], data[ix], marker="x", color="k", s=128.0)
        # end
    end
    # modify y axis to reflect truncated data
    ax.yaxis.set_ticks(0:2:26)
    ax.yaxis.set_ticklabels([string.(0:2:24); ">26"])

    # set labels
    ax.set_title(titlestr[reindex[k]])
    ax.set_xlabel("Proportion initially infected (%)")
    ax.set_ylabel("Weeks after reopening")

    # panel labels
    ax.text(-0.04, 1.075, string('A'+(reindex[k]-1)), transform=ax.transAxes,
      fontsize=16, fontweight="bold", va="top", ha="right")
end

fig.legend(
    title="Contact multiplier",
    [L"\times~" * "$(c)" for c in multiplier_range],
    bbox_to_anchor=(0.35, -0.05, 1.02, 0),
    loc="lower left",
    borderaxespad=0,
    frameon=false,
    ncol=3)
fig.tight_layout()

PyPlot.savefig("../figures/Fig3.png", dpi=300, bbox_inches="tight")
PyPlot.savefig("../figures/Fig3.tiff", dpi=300, bbox_inches="tight")

fig

### Fig S4: Interactions between demography and proportion infected

In [ ]:
children_percent = range(0, 60, length=61)
children_infected = range(0.0, 50, length=51)
infected_range = [1e-1, 1e0, 5e0, 1e1]

m = length(children_percent)
n = length(children_infected)

In [ ]:
# scenario parameters
nclasses = 2
ncohorts = 2
run_parallel = true
t_thresh = [zeros(m, n) for _ in 1:ncohorts+run_parallel, _ in 1:length(infected_range)]

for (l, initial_infected) in enumerate(infected_range), k in 1:ncohorts+run_parallel
    # generate problems with different parameter values
    prob_matrix = Array{Any}(undef, m, n)
    cohort_count = min(ncohorts, k)

    for (j, c_infected) in enumerate(children_infected), (i, c_percent) in enumerate(children_percent)
        beta = β0 * F
        demos = [c_percent, 100-c_percent]
        prob_matrix[i,j] = make_model_instance(nclasses, cohort_count, tspan, beta, params,
            mult=multiplier,
            demographics=demos,
            total_infected=initial_infected,
            infected=[c_infected, 100-c_infected],
        )
    end

    #
    # solve the ODE for different parameter values
    # save data for each day
    #
    if k ≤ ncohorts
        message = "Simulation for $(k) cohort(s), $(initial_infected)% infected "
    else
        message = "Simulation for 2 parallel cohorts, $(initial_infected)% infected "
    end

    @showprogress message for (ix, prob) in enumerate(prob_matrix)
        if k ≤ ncohorts
            isrotating = true
        else
            isrotating = false
            # only the first cohort attends school in person
            cohort_β = prob.p[4]

            # cohort 1 --- in-person
            ratef = cohort_β[1][1,1]
            cohort_β[1][1,1] = TransmissionRate(ratef.cmax, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

            # cohort 2 --- remote learning
            ratef = cohort_β[2][1,1]
            cohort_β[2][1,1] = TransmissionRate(ratef.cmin, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)
        end

        detected = sizehint!(Float64[], length(test_times))
        cb_test = make_testing_policy_callback(test_times, detected,
            sensitivity=sensitivity,
            isrotating=isrotating,
        )
        sol = solve(prob, callback=cb_test, userdata=Dict(:school_closed=>false))

        # adjust for population demographics
        demos = dropdims(sum(prob.u0, dims=(1,3)), dims=(1,3))

        # extract 14-day window infection data
        offset = window-1
        x = [ sum(detected[max(1,i-window):i]) for i in eachindex(detected) ]

        # find time to hit threshold
        index = findfirst(≥(demos[1]/sum(demos)*threshold), x)
        t_thresh[k,l][ix] = isnothing(index) ? Inf : test_times[index]
    end
end

In [ ]:
rcParams["axes.grid"] = false
rcParams["axes.facecolor"] = "1.0"

nrows = ncohorts+run_parallel
ncols = length(infected_range)
fig, axes = subplots(nrows=nrows, ncols=ncols, figsize=[4*ncols, 4*nrows])

# add separate axis to unify labels
fig.add_subplot(frameon=false)
plt.tick_params(labelcolor="none", top=false, bottom=false, left=false, right=false)
plt.xlabel("Infected children at reopening (%)")
plt.ylabel("Children in population (%)")

img = []
for (k, ax) in enumerate(axes)
    iii = ax.contourf(children_infected, children_percent, round.(t_thresh[k] ./ 7),
        levels=0:2:26,
        extend="max",
        vmin=0,
        vmax=26,
        cmap=get_cmap("viridis_r")
    )
    push!(img, iii)
end

# add labels for columns
for (l, percent) in enumerate(infected_range)
    ax = axes[nrows*(l-1)+1]
    ax.set_title("$(percent)% infected at reopening")
end

# add labels for rows
for k in 1:size(t_thresh,1)
    ax = axes[k]
    if k ≤ ncohorts
        label = string("$(k) ", k == 1 ? "cohort" : "cohorts (rotating)")
    else
        label = "$(ncohorts) cohorts (parallel)"
    end
    ypos = k == 1 ? 0.4 : 0.2
    ax.text(-0.3, ypos, label, rotation=90, transform=ax.transAxes, fontsize=16)
end

# add color bar
cb_ax = fig.add_axes([0.27, 0.0, 0.5, 0.02])
cbar = fig.colorbar(img[1], cax=cb_ax, orientation="horizontal", label="weeks since reopening")#, norm=cnorm, boundaries=bounds)
cbar.set_ticks(0:2:26)

fig.tight_layout()

PyPlot.savefig("../figures/S4_fig.png", dpi=300, bbox_inches="tight")
PyPlot.savefig("../figures/S4_fig.tiff", dpi=300, bbox_inches="tight")

fig

# Figure 4: Threshold policy

Single cohort, close down schools upon hitting 5% infections in children

In [ ]:
# not really needed anymore
scale_percent(t, x) = (t, scale_percent(x))
scale_percent(x) = x * 100

# recording data
function record_figure_data!(data, sol, detected, threshold, window, test_times, demographics)
    # shorthand
    q = demographics / sum(demographics)
    ts = sol.t
    u = sol.u

    # find threshold time
    x = [ sum(detected[max(1,i-window):i]) for i in eachindex(detected) ]
    index = findfirst(≥(q[1]*threshold), x)
    t_thresh = isnothing(index) ? sol.prob.tspan[end] : test_times[index]

    # time points in solution
    push!(data.ts, sol.t)

    # child data
    push!(data.child_susceptible, scale_percent.(susceptible_child_fraction.(u)) ./ q[1])
    push!(data.child_infected,    scale_percent.(infected_child_fraction.(u)) ./q[1])
    push!(data.child_removed,     scale_percent.(removed_child_fraction.(u)) ./ q[1])
    push!(data.child_detected,    scale_percent.(x) ./ q[1])

    push!(data.t_thresh,          t_thresh)

    push!(data.adult_susceptible, scale_percent.(susceptible_adult_fraction.(u)) ./ q[2])
    push!(data.adult_infected,    scale_percent.(infected_adult_fraction.(u)) ./ q[2])
    push!(data.adult_removed,     scale_percent.(removed_adult_fraction.(u)) ./ q[2])

    return nothing
end

In [ ]:
# policy parameters
sensitivity_tested = [1.0, 0.5]

# simulation parameters
nclasses = 2
ncohorts = 1
total_infected = 2.0

# allocate outputs
data = (
    ts                = Vector{Float64}[], # time axis
    child_infected    = Vector{Float64}[], # sum_k I_1k
    child_susceptible = Vector{Float64}[], # sum_k S_1k
    child_removed     = Vector{Float64}[], # sum_k R_1k
    child_detected    = Vector{Float64}[], # 14-window criterion
    t_thresh          = Float64[],         # threshold time
    adult_infected    = Vector{Float64}[], # sum_k I_2k
    adult_susceptible = Vector{Float64}[], # sum_k S_2k
    adult_removed     = Vector{Float64}[], # sum_k R_2k
)

# closure to make things easier
record_figure_data!(data, sol, detected) = record_figure_data!(data, sol, detected, threshold, window, test_times, demographics)

#
# TOP: no intervention
#
detected_cases = Float64[]

prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)
sol = solve(prob)

# record data
record_figure_data!(data, sol, detected_cases)

#
# MIDDLE: with intervention, perfect detection
#
detected_cases = Float64[]
cb_test, cb_threshold, cbs = make_threshold_policy_callbacks(test_times, detected_cases,
    sensitivity=sensitivity_tested[1],
    isrotating=true,
    threshold=threshold,
    window=window,
)
prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)
sol = solve(prob, callback=cbs, userdata=Dict(:school_closed=>false))

# record data
record_figure_data!(data, sol, detected_cases)

#
# BOTTOM: with intervention, 50% detection
#
detected_cases = Float64[]
cb_test, cb_threshold, cbs = make_threshold_policy_callbacks(test_times, detected_cases,
    sensitivity=sensitivity_tested[2],
    isrotating=true,
    threshold=threshold,
    window=window,
)
prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)
sol = solve(prob, callback=cbs, userdata=Dict(:school_closed=>false))

# record data
record_figure_data!(data, sol, detected_cases)

In [ ]:
# background
rcParams["axes.grid"] = false
rcParams["axes.facecolor"] = "0.90"

fig, ax = subplots(figsize=[3*3.0, 2*4], nrows=4)

fig.add_subplot(frameon=false)
plt.tick_params(labelcolor="none", top=false, bottom=false, left=false, right=false)
plt.xlabel("Weeks after reopening")
plt.ylabel("Proportion (%)")

common_xticks = 0:14:tspan[end] # show every two weeks
common_xtickl = string.(Int.(common_xticks .÷ 7))

ymin = 0.0
ymax = 8.0

##### no intervention ####

for i in eachindex(data.ts)
    # 1,1 -- infections in children
    x = data.child_detected[i]
    ax[1].plot(data.ts[i], data.child_infected[i], "-", color="C$(i-1)", lw=1.5)
    ax[1].plot(test_times[1:length(x)], x, "--", color="C$(i-1)", lw=3)
    (i > 1) && ax[1].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 1,2 -- susceptible & removed in children
    ax[2].plot(data.ts[i], data.child_susceptible[i], "-", color="C$(i-1)", lw=1.5)

    # 2,1 -- infections in adults
    ax[3].plot(data.ts[i], data.adult_infected[i], "-", color="C$(i-1)", lw=1.5)

    # 2,2 -- susceptible & removed in adults
    ax[4].plot(data.ts[i], data.child_susceptible[i], "-", color="C$(i-1)", lw=1.5)
end

for i in eachindex(ax)
    ax[i].xaxis.set_ticks(common_xticks)
    ax[i].xaxis.set_ticklabels(common_xtickl)
    if (i % 2 == 0)
        ax[i].set_ylim(20, 105)
        ax[i].set_yticks(20:20:100)
    end
    if (i % 2 == 1)
        ax[i].set_ylim(0, 10.0)
        ax[i].set_yticks(0:2:10)
    end
end

ax[1].set_title("Infections (children)")
ax[2].set_title("Susceptibles (children)")
ax[3].set_title("Infections (adults)")
ax[4].set_title("Susceptibles (adults)")

# add figure legend
artist_c0 = plt.Line2D((0,1), (0,0), color="C0")
artist_c1 = plt.Line2D((0,1), (0,0), color="C1")
artist_c2 = plt.Line2D((0,1), (0,0), color="C2")

artist_solid = plt.Line2D((0,1), (0,0), color="k", linestyle="-")
artist_dash = plt.Line2D((0,1), (0,0), color="k", linestyle="--")

fig.legend(
    (artist_c0, artist_c1, artist_c2, artist_solid, artist_dash),
    ("no intervention", "100% detection", "50% detection", latexstring("total ", L"I(t)"), "detected"),
    bbox_to_anchor=(0.025,-0.0,0.5,0),
    loc="lower left",
    borderaxespad=0.0,
    frameon=false,
    ncol=5)
fig.subplots_adjust(bottom=0.09)

# add labels to each subplot
for i in eachindex(ax)
    ax[i].text(-0.01, 1.25, string('A'+(i-1)), transform=ax[i].transAxes,
      fontsize=16, fontweight="bold", va="top", ha="right")
    ax[i].grid(axis="y")
end

fig.tight_layout()

PyPlot.savefig("../figures/Fig4.png", dpi=300, bbox_inches="tight")
PyPlot.savefig("../figures/Fig4.tiff", dpi=300, bbox_inches="tight")

fig

# Appendix

In [ ]:
PyPlot.close_figs()

In [ ]:
using InteractiveUtils; versioninfo()

In [ ]:
using Pkg; Pkg.status()